In [1]:
import gzip
import nltk
import pandas as pd
import numpy as np
import pickle
import multiprocessing

from collections import Counter
from nltk.corpus import stopwords

In [2]:
import utils as my_utils

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [4]:
def process_df(df):
    df['cleaned'] = my_utils.preprocess(df['reviewText'])
    df['text'] = df['cleaned'].apply(lambda x: " ".join(x))
    return df

In [5]:
df = getDF('datasets/reviews_Musical_Instruments_5.json.gz')
df.shape

(10261, 9)

In [6]:
n_cores = 2

In [7]:
n = int(df.shape[0]/n_cores)
list_df = [df[i:i+n] for i in range(0, df.shape[0],n)]

pool = multiprocessing.Pool(n_cores)
processed_list_df = pool.map(process_df, list_df)
pool.close()

df = pd.concat(processed_list_df)
df.shape

(10261, 11)

In [8]:
df.to_pickle("datasets/datadf_amazon_musical")